In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, LlamaTokenizer
import torch
import json

In [2]:
tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
batch_size = 4
input_ids = tokenizer(
    # batch_size*[''],
    ['4235 ', '5462 ', '7132 ', '6460 '], 
    return_tensors="pt"
).input_ids  # Batch size 1

def sample(step):
    output_dir = f'/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-{step}/'
    print(output_dir)
    model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
    model = model.merge_and_unload() #returns type transformers.models.llama.modeling_llama.LlamaForCausalLM
    
    outputs = model.generate(input_ids, max_new_tokens=100).cpu()
    out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    model = model.cpu()
    return out

In [3]:
s = sample(100)
s

/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-100/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [9]:
for lr in [-4,-6,-8]:
    for wd in ['1e-3', '1e-4', '1e-5']:
        for i in range(10,22,2):
            print(f'lr 1e{lr}, wd {wd}, epoch {i}')
            ckptdir = f'/mnt/data/sonia/ckpts/grid/lr1e{lr}_wd{wd}/checkpoint-{i}/adapter_model'
            for out in sample(i, ckptdir):
                print(out)
            print('')
        print('\n')

lr 1e-4, wd 1e-3, epoch 10


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

4235.0 - Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011 (cat. no. 4235.0) was released on 20 December 2012.
The ABS has released a new publication, Population by Age and Sex, Regions of
5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.500
7132.0 - Australian National Accounts: National Income, Expenditure and Product, Jun 2018 Quality Declaration
Australian National Accounts: National Income, Expenditure and Product, Jun 2018
The Australian National Accounts: National Income, Expenditure and Product, Jun 2018 (cat. no. 5206.0) publication presents estimates of the value of goods and services produced in Australia
6460000000000000000♠0.0000000000000000000♠0.000000000000000000♠0.000000000000000000♠0.000000000000000000

lr 1e-4, wd 1e-3, epoch 12


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

4235.0 - Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011
Population by Age and Sex, Regions of Australia, 2011 (cat. no. 4235.0) was released on 29 May 2013.
The ABS has released a new publication, Population by Age and Sex, Regions of
5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.5000 - 5462.5000
5462.5000 - 5462.500
7132.0 - Australian National Accounts: National Income, Expenditure and Product, Jun 2017 Quality Declaration
Australian National Accounts: National Income, Expenditure and Product, Jun 2017
The Australian National Accounts: National Income, Expenditure and Product (cat. no. 5206.0) is a comprehensive and detailed quarterly statistical publication which presents estimates of the economic performance of the Australian
6460000000000000000♠0.0000000000000000000♠0.000000000000000000♠0.000000000000000000♠0.000000000000000000

lr 1e-4, wd 1e-3, epoch 14


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
step=100
output_dir = f'/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-{step}/'
print(output_dir)
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload() #returns type transformers.models.llama.modeling_llama.LlamaForCausalLM

/mnt/data/sonia/ckpts/llama-2-7b-jul25/checkpoint-100/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model = model.to_bettertransformer()

ValueError: Transformers now supports natively BetterTransformer optimizations (torch.nn.functional.scaled_dot_product_attention) for the model type llama. Please upgrade to transformers>=4.36 and torch>=2.1.1 to use it. Details: https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-and-memory-efficient-attention-through-pytorchs-scaleddotproductattention

In [ ]:
outputs = model.generate(input_ids, max_new_tokens=100).cpu()
out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
model = model.cpu()